# Fasttext
- https://radimrehurek.com/gensim/models/fasttext.html

In [ ]:
# !pip install fasttext

# Fasttext-train

In [ ]:
import pandas as pd
import fasttext
import matplotlib.pyplot as plt

# 변수 지정
model_path = './fasttext_save/fasttext_model_v1.bin'
epoch = 20  # 최대 epoch 수
lr = 0.5
wordNgrams = 2
minCount = 1
patience = 3  # Early stopping patience

# 데이터 로드 및 전처리
train_df = pd.read_excel('./data/train_set.xlsx')
valid_df = pd.read_excel('./data/valid_label_set.xlsx')

# 라벨 변환
train_df.label = train_df.label.map({'yes': 1, 'no': 0})
valid_df.label = valid_df.label.map({'yes': 1, 'no': 0})

# FastText 형식으로 변환
def convert_to_fasttext_format(df, filename):
    with open(filename, 'w') as f:
        for _, row in df.iterrows():
            label = f"__label__{row['label']}"
            text = row['question']
            f.write(f"{label} {text}\n")

convert_to_fasttext_format(train_df, 'train_fasttext.txt')
convert_to_fasttext_format(valid_df, 'valid_fasttext.txt')

# 모델 학습 및 평가 기록 저장
def train_and_evaluate(train_file, valid_file):
    train_accuracies = []
    valid_accuracies = []
    train_losses = []
    valid_losses = []
    model = None  # 모델 변수를 함수 밖에서도 사용할 수 있도록 설정
    
    best_valid_loss = float('inf')
    epochs_no_improve = 0
    
    for ep in range(1, epoch + 1):
        model = fasttext.train_supervised(input=train_file, epoch=ep, lr=lr, wordNgrams=wordNgrams, verbose=2, minCount=minCount)
        
        # Train set evaluation
        train_result = model.test(train_file)
        train_accuracy = train_result[1]
        train_loss = train_result[2]

        # Valid set evaluation
        valid_result = model.test(valid_file)
        valid_accuracy = valid_result[1]
        valid_loss = valid_result[2]
        
        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        print(f"Epoch {ep}/{epoch} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

        # Early stopping check
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            epochs_no_improve = 0
            model.save_model(model_path)  # Best model 저장
            print(f"Model saved to {model_path}")
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {ep}")
            break

    return train_accuracies, valid_accuracies, train_losses, valid_losses

# 모델 평가항목
train_accuracies, valid_accuracies, train_losses, valid_losses = train_and_evaluate('train_fasttext.txt', 'valid_fasttext.txt')

# 시각화
epochs = range(1, len(train_accuracies) + 1)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracies, 'bo-', label='Train Accuracy')
plt.plot(epochs, valid_accuracies, 'ro-', label='Valid Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train and Valid Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_losses, 'bo-', label='Train Loss')
plt.plot(epochs, valid_losses, 'ro-', label='Valid Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Valid Loss')
plt.legend()

plt.tight_layout()
plt.show()


# Fasttext-valid


In [ ]:
import pandas as pd
import fasttext

# 사용할 모델 지정
model_path = './fasttext_save/fasttext_model_v1.bin'

# FastText 모델 로드
model = fasttext.load_model(model_path)

# 라벨이 없는 검증 데이터셋 로드
valid_df = pd.read_excel('./data/valid_set.xlsx')

# FastText를 사용한 라벨 예측 함수 정의
def predict_labels(model, texts):
    labels, probabilities = model.predict(texts)
    # 예측된 라벨에서 "__label__" 부분 제거
    labels = [label[0].replace('__label__', '') for label in labels]
    return labels

# 검증 데이터셋에서 질문 텍스트 추출
valid_texts = valid_df['question'].tolist()

# 라벨 예측
predicted_labels = predict_labels(model, valid_texts)

# 예측된 라벨을 데이터프레임에 추가
valid_df['predicted_label'] = predicted_labels

# 예측된 라벨을 파일로 저장
valid_df.to_excel('./data/valid_set_with_labels.xlsx', index=False)

print("라벨 예측 완료 및 파일 저장 완료")


## 임계값 기준 라벨링

In [ ]:
import pandas as pd
import fasttext

# 사용할 모델 지정
model_path = './fasttext_save/fasttext_model_v1.bin'

# 임계값
threshold = 0.7

# FastText 모델 로드
model = fasttext.load_model(model_path)

# 라벨이 없는 검증 데이터셋 로드
valid_df = pd.read_excel('./data/valid_set.xlsx')

# FastText를 사용한 라벨 예측 함수 정의
def predict_labels(model, texts, threshold):
    labels, probabilities = model.predict(texts, k=2)  # 두 개의 라벨 확률을 반환하도록 설정
    adjusted_labels = []
    for label, prob in zip(labels, probabilities):
        # 각 라벨의 확률을 가져오기
        label_prob_dict = dict(zip(label, prob))
        label_0_prob = label_prob_dict.get('__label__0', 0)
        label_1_prob = label_prob_dict.get('__label__1', 0)
        
        # 임계값에 따라 라벨을 예측하는 로직
        if label_1_prob >= threshold:
            adjusted_labels.append(1)
        elif label_0_prob >= threshold and label_0_prob > label_1_prob:
            adjusted_labels.append(0)
        else:
            # 두 라벨 모두 임계값 미달인 경우 더 높은 확률을 가진 라벨로 설정
            adjusted_labels.append(0 if label_0_prob > label_1_prob else 1)
    return adjusted_labels, [max(prob) for prob in probabilities]

# 검증 데이터셋에서 질문 텍스트 추출
valid_texts = valid_df['question'].tolist()

# 라벨 예측
predicted_labels, probabilities = predict_labels(model, valid_texts, threshold)

# 예측된 라벨과 확률값을 데이터프레임에 추가
valid_df['predicted_label_thres'] = predicted_labels
valid_df['probability'] = probabilities

# 예측된 라벨을 파일로 저장
valid_df.to_excel('./data/valid_set_with_labels_by_threshold.xlsx', index=False)

print("라벨 예측 완료 및 파일 저장 완료")


## 특정 확률값 이상 라벨링

In [ ]:
import numpy as np
import pandas as pd
import fasttext

# 사용할 모델 지정
model_path = './fasttext_save/fasttext_model_v1.bin'

# 특정 확률값
min_probability = 0.5

# FastText 모델 로드
model = fasttext.load_model(model_path)

# 라벨이 없는 검증 데이터셋 로드
valid_df = pd.read_excel('./data/valid_set.xlsx')

# FastText를 사용한 라벨 예측 함수 정의
def predict_labels_by_prob(model, texts, min_probability):
    labels, probabilities = model.predict(texts, k=2)  # 두 개의 라벨 확률을 반환하도록 설정
    adjusted_labels = []
    for label, prob in zip(labels, probabilities):
        max_prob = max(prob)
        # 확률 값이 min_probability 이상인 경우 라벨 설정
        if max_prob >= min_probability:
            # 높은 확률 값을 가진 라벨을 선택
            high_prob_label = label[np.argmax(prob)]
            adjusted_labels.append(0 if high_prob_label == '__label__0' else 1)
        else:
            # 반대 라벨을 설정
            low_prob_label = label[np.argmin(prob)]
            adjusted_labels.append(0 if low_prob_label == '__label__0' else 1)
    return adjusted_labels, [max(prob) for prob in probabilities]

# 검증 데이터셋에서 질문 텍스트 추출
valid_texts = valid_df['question'].tolist()

# 라벨 예측
predicted_labels, probabilities = predict_labels_by_prob(model, valid_texts, min_probability)

# 예측된 라벨과 확률값을 데이터프레임에 추가
valid_df['predicted_label_prob'] = predicted_labels
valid_df['probability'] = probabilities

# 예측된 라벨을 파일로 저장
valid_df.to_excel('./data/valid_set_with_labels_by_prob.xlsx', index=False)

print("라벨 예측 완료 및 파일 저장 완료")


## 임계값과 특정 확률값 이상 라벨링

In [ ]:
import pandas as pd
import fasttext

# 사용할 모델 지정
model_path = './fasttext_save/fasttext_model_v1.bin'

# 임계값, 최소 확률값
threshold = 0.7
min_probability = 0.5

# FastText 모델 로드
model = fasttext.load_model(model_path)

# 라벨이 없는 검증 데이터셋 로드
valid_df = pd.read_excel('./data/valid_set.xlsx')

# FastText를 사용한 라벨 예측 함수 정의
def predict_labels_by_threshold_and_prob(model, texts, threshold, min_probability):
    labels, probabilities = model.predict(texts, k=2)  # 두 개의 라벨 확률을 반환하도록 설정
    adjusted_labels = []
    for label, prob in zip(labels, probabilities):
        # 각 라벨의 확률을 가져오기
        label_prob_dict = dict(zip(label, prob))
        label_0_prob = label_prob_dict.get('__label__0', 0)
        label_1_prob = label_prob_dict.get('__label__1', 0)
        max_prob = max(prob)
        
        # 임계값과 확률 값 조건 모두 만족하는 경우 라벨 설정
        if max_prob >= min_probability:
            if label_1_prob >= threshold and label_1_prob > label_0_prob:
                adjusted_labels.append(1)
            elif label_0_prob >= threshold and label_0_prob > label_1_prob:
                adjusted_labels.append(0)
            else:
                # 두 라벨 모두 임계값 미달인 경우 더 높은 확률을 가진 라벨로 설정
                adjusted_labels.append(0 if label_0_prob > label_1_prob else 1)
        else:
            # 확률 값이 min_probability 미달인 경우 라벨 설정 안함 (예: -1로 설정)
            adjusted_labels.append(-1)
    return adjusted_labels, [max(prob) for prob in probabilities]

# 검증 데이터셋에서 질문 텍스트 추출
valid_texts = valid_df['question'].tolist()

# 라벨 예측
predicted_labels, probabilities = predict_labels_by_threshold_and_prob(model, valid_texts, threshold, min_probability)

# 예측된 라벨과 확률값을 데이터프레임에 추가
valid_df['predicted_label_all'] = predicted_labels
valid_df['probability'] = probabilities

# 예측된 라벨을 파일로 저장
valid_df.to_excel('./data/valid_set_with_labels_by_threshold_and_prob.xlsx', index=False)

print("라벨 예측 완료 및 파일 저장 완료")
